In [5]:
import tensorflow as tf
import numpy as np
import skflow

In [3]:
a = tf.placeholder("float") # 创建符号变量
b = tf.placeholder("float") 

y = tf.mul(a, b) # 乘法操作，作用在符号变量上。

sess = tf.Session() # 创建会话，计算符号变量表达式

a1 = 4
b1 = 5
print "%f + %f = %f"%(4, 5,  sess.run(y, feed_dict={a: a1, b: b1}) )

4.000000 + 5.000000 = 20.000000


In [4]:
trX = np.linspace(-1, 1, 101)
print trX

[-1.   -0.98 -0.96 -0.94 -0.92 -0.9  -0.88 -0.86 -0.84 -0.82 -0.8  -0.78
 -0.76 -0.74 -0.72 -0.7  -0.68 -0.66 -0.64 -0.62 -0.6  -0.58 -0.56 -0.54
 -0.52 -0.5  -0.48 -0.46 -0.44 -0.42 -0.4  -0.38 -0.36 -0.34 -0.32 -0.3
 -0.28 -0.26 -0.24 -0.22 -0.2  -0.18 -0.16 -0.14 -0.12 -0.1  -0.08 -0.06
 -0.04 -0.02  0.    0.02  0.04  0.06  0.08  0.1   0.12  0.14  0.16  0.18
  0.2   0.22  0.24  0.26  0.28  0.3   0.32  0.34  0.36  0.38  0.4   0.42
  0.44  0.46  0.48  0.5   0.52  0.54  0.56  0.58  0.6   0.62  0.64  0.66
  0.68  0.7   0.72  0.74  0.76  0.78  0.8   0.82  0.84  0.86  0.88  0.9
  0.92  0.94  0.96  0.98  1.  ]


In [5]:
# 生成训练数据 + 噪声，下面为了拟合 $$ Y = 2X $$ 
trX = np.linspace(-1, 1, 101)
trY = 2 * trX + np.random.randn(*trX.shape) * 0.33 # y=2x，但是加入了噪声

X = tf.placeholder("float") #输入输出符号变量
Y = tf.placeholder("float")

# 定义模型
def model(X, w):
    return tf.mul(X, w) # 线性回归只需要调用乘法操作即可。

# 模型权重 W 用变量表示
w = tf.Variable(0.0, name="weights") # 共享变量
y_model = model(X, w)

# 定义损失函数
cost = (tf.pow(Y-y_model, 2)) # 平方损失函数

# 构建优化器，最小化损失函数。
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(cost) 

# 构建会话
sess = tf.Session()

# 初始化所有的符号共享变量
init = tf.initialize_all_variables() 

# 运行会话
sess.run(init)

# 迭代训练
for i in range(100):
    for (x, y) in zip(trX, trY): 
        sess.run(train_op, feed_dict={X: x, Y: y})
# 打印权重w
print(sess.run(w))  

2.1181


In [6]:
from tensorflow.examples.tutorials.mnist import input_data
# 初始化权重w
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

# 定义模型
def model(X, w):
    return tf.matmul(X, w) 

# 获取mnist 数据
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

# 定义占位符变量
X = tf.placeholder("float", [None, 784]) 
Y = tf.placeholder("float", [None, 10])

w = init_weights([784, 10]) 
py_x = model(X, w)

# 定义损失函数，交叉熵损失函数
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y)) 

# 训练操作，最小化损失函数
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost) 

# 预测操作，
predict_op = tf.argmax(py_x, 1) 

# 定义会话
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

# 调用多次梯度下降
for i in range(100):
    # 训练，每个batch，
    for start, end in zip(range(0, len(trX), 128), range(128, len(trX), 128)):
        sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})
    # 测试，每个epoch
    print i, np.mean(np.argmax(teY, axis=1) ==
                     sess.run(predict_op, feed_dict={X: teX, Y: teY}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
0 0.8831
1 0.8967
2 0.9037
3 0.9073
4 0.9099
5 0.9107
6 0.912
7 0.9128
8 0.9147
9 0.9156
10 0.9158
11 0.9159
12 0.9168
13 0.9169
14 0.917
15 0.9175
16 0.9179
17 0.9183
18 0.9187
19 0.9188
20 0.9194
21 0.9197
22 0.9202
23 0.92
24 0.9203
25 0.9203
26 0.9204
27 0.9203
28 0.9204
29 0.9206
30 0.9211
31 0.921
32 0.921
33 0.9211
34 0.9209
35 0.921
36 0.9212
37 0.9214
38 0.9216
39 0.9217
40 0.9214
41 0.9218
42 0.9217
43 0.9218
44 0.9217
45 0.9217
46 0.9217
47 0.922
48 0.9218
49 0.9217
50 0.9217
51 0.9218
52 0.9219
53 0.922
54 0.9222
55 0.9221
56 0.9224
57 0.9225
58 0.9227
59 0.9229
60 0.9229
61 0.923
62 0.9231
63 0.9231
64 0.9231
65 0.9231
66 0.9232
67 0.9232
68 0.9233
69 0.9234
70 0.9234
71 0.9234
72 0.9234
73 0.9235
74 0.9234
75 0.9235
76 0.9235
77 0.9234
78 0.9232
79 0.9235
80 0.9235
81 0.9235
82 0.9

In [7]:
persontype = np.dtype({
        'names': ['name', 'age', 'weight'], 
        'formats': ['S32', 'i', 'f']
    })
a = np.array([("Zhang", 32, 75.5), ("Wang", 24, 65.2)], dtype=persontype)
%timeit a[1]
%timeit a['name']
%timeit a[1]['name']
%timeit a['name'][1]

The slowest run took 26.31 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 109 ns per loop
The slowest run took 38.32 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 156 ns per loop
The slowest run took 34.77 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 693 ns per loop
The slowest run took 10.80 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 265 ns per loop


In [8]:
import tensorflow as tf
import numpy as np

# 使用 NumPy 生成假数据(phony data), 总共 100 个点.
x_data = np.float32(np.random.rand(2, 100)) # 随机输入
y_data = np.dot([0.100, 0.200], x_data) + 0.300

# 构造一个线性模型
# 
b = tf.Variable(tf.zeros([1]))
W = tf.Variable(tf.random_uniform([1, 2], -1.0, 1.0))
y = tf.matmul(W, x_data) + b

# 最小化方差
loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

# 初始化变量
init = tf.initialize_all_variables()

# 启动图 (graph)
sess = tf.Session()
sess.run(init)

# 拟合平面
for step in xrange(0, 201):
    sess.run(train)
    if step % 20 == 0:
        print step, sess.run(W), sess.run(b)

# 得到最佳拟合结果 W: [[0.100  0.200]], b: [0.300]

0 [[ 0.5408504  -0.39501327]] [ 0.91111481]
20 [[ 0.14222436 -0.00550158]] [ 0.39396366]
40 [[ 0.10332326  0.14406639]] [ 0.32961255]
60 [[ 0.09895143  0.18444969]] [ 0.30920762]
80 [[ 0.09923692  0.19559805]] [ 0.30283692]
100 [[ 0.0996725   0.19873527]] [ 0.3008686]
120 [[ 0.09988005  0.19963233]] [ 0.30026475]
140 [[ 0.09995924  0.19989212]] [ 0.30008042]
160 [[ 0.0999867   0.19996814]] [ 0.30002436]
180 [[ 0.09999577  0.19999054]] [ 0.30000737]
200 [[ 0.09999867  0.19999719]] [ 0.30000225]


In [16]:
import pandas
data = pandas.read_csv('../tf_examples/data/titanic_train.csv')
data.shape

(891, 12)

In [17]:
data.columns

Index([u'PassengerId', u'Survived', u'Pclass', u'Name', u'Sex', u'Age',
       u'SibSp', u'Parch', u'Ticket', u'Fare', u'Cabin', u'Embarked'],
      dtype='object')

In [18]:
data[:1]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


In [27]:
import random
import pandas
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.utils import check_array
from sklearn.cross_validation import train_test_split

import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib import learn

y, X = data['Survived'], data[['Age', 'SibSp', 'Fare']].fillna(0)
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2, random_state=42)
lr = LogisticRegression()
lr.fit(X_train, y_train)
print accuracy_score(lr.predict(X_test), y_test)

0.664804469274


In [28]:
# 创建一个常量 op, 产生一个 1x2 矩阵. 这个 op 被作为一个节点
# 加到默认图中.
#
# 构造器的返回值代表该常量 op 的返回值.
matrix1 = tf.constant([[3., 3.]])

# 创建另外一个常量 op, 产生一个 2x1 矩阵.
matrix2 = tf.constant([[2.],[2.]])

# 创建一个矩阵乘法 matmul op , 把 'matrix1' 和 'matrix2' 作为输入.
# 返回值 'product' 代表矩阵乘法的结果.
product = tf.matmul(matrix1, matrix2)
# 启动默认图.
sess = tf.Session()

# 调用 sess 的 'run()' 方法来执行矩阵乘法 op, 传入 'product' 作为该方法的参数. 
# 上面提到, 'product' 代表了矩阵乘法 op 的输出, 传入它是向方法表明, 我们希望取回
# 矩阵乘法 op 的输出.
#
# 整个执行过程是自动化的, 会话负责传递 op 所需的全部输入. op 通常是并发执行的.
# 
# 函数调用 'run(product)' 触发了图中三个 op (两个常量 op 和一个矩阵乘法 op) 的执行.
#
# 返回值 'result' 是一个 numpy `ndarray` 对象.
result = sess.run(product)
print result
# ==> [[ 12.]]

# 任务完成, 关闭会话.
sess.close()

[[ 12.]]
